In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('all')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Reviews.csv')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [3]:
#Очистка данных
import re
from datetime import datetime as dt
df_not_none = df.dropna()
df_not_none['Time'] = df_not_none['Time'].apply(lambda x: dt.fromtimestamp(x))
df_not_none_no_dupl = df_not_none.drop_duplicates()
df_not_none_no_dupl = df_not_none_no_dupl[~(df_not_none_no_dupl['HelpfulnessNumerator'] > df_not_none_no_dupl['HelpfulnessDenominator'])]
df_not_none_no_dupl = df_not_none_no_dupl.drop(columns=['ProfileName'], axis = 1).reset_index(drop=True)
df_not_none_no_dupl = df_not_none_no_dupl.drop_duplicates(subset=['ProductId', 'UserId', 'Text'])
df_not_none_no_dupl = df_not_none_no_dupl.drop(columns=['HelpfulnessDenominator'], axis = 1).reset_index(drop=True)
df_not_none_no_dupl = df_not_none_no_dupl.drop(columns=['HelpfulnessNumerator'], axis = 1).reset_index(drop=True)
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'\d+', '', text)  # Удаление цифр
    stop_words = set(stopwords.words('english'))  # Стоп-слова на английском
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text
df_not_none_no_dupl['Text'] = df_not_none_no_dupl['Text'].apply(lambda x: clean_text(x))

<ipython-input-3-a37702d132ff>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_none['Time'] = df_not_none['Time'].apply(lambda x: dt.fromtimestamp(x))


In [5]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.7 MB/s eta 0:00:00


In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
random_sample_df = df_not_none_no_dupl.sample(n=10000, random_state=42)

# Инициализируем анализатор VADER
analyzer = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    # Получаем тональность текста
    scores = analyzer.polarity_scores(text)
    # Анализируем compound score: >0 - позитивный, <0 - негативный
    if scores['compound'] > 0:
        return 1
    elif scores['compound'] <= 0:
        return 0


# Применяем функцию к столбцу с отзывами
random_sample_df['Positive'] = random_sample_df['Text'].apply(get_vader_sentiment)

In [15]:
random_sample_df.head()


,Id,ProductId,UserId,Score,Time,Summary,Text,Positive
165363,165795,B000CPZSC8,A3CK0SW0H2HGWX,5,2012-02-20,Best Value Vanilla!,vanilla pricing fluctuates pretty low right st...,1
312513,313285,B003WAX3H2,A2S3MH8WNMG0LE,5,2010-11-16,great variety,excellent selection coffee help decide type fl...,1
270563,271264,B001IZHZGS,A3QKH94YUG0IEI,5,2012-08-27,Great,great fresh came fast love gum tastes like haw...,1
559088,560441,B0005ZZ4CE,AKYX5JND0LTL8,4,2010-09-14,"Best sauce, Worst price",ortega medium taco sauce best sauce ever entir...,1
136056,136380,B006Q820X0,#oc-R192YGWMM86YIU,5,2012-04-25,UPDATED Review Problem resolved,since new product released feedback usually lo...,1


In [16]:
# create preprocess_text function
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

# apply the function df
random_sample_df['Text'] = random_sample_df['Text'].apply(preprocess_text)
random_sample_df.head()

,Id,ProductId,UserId,Score,Time,Summary,Text,Positive
165363,165795,B000CPZSC8,A3CK0SW0H2HGWX,5,2012-02-20,Best Value Vanilla!,vanilla pricing fluctuates pretty low right st...,1
312513,313285,B003WAX3H2,A2S3MH8WNMG0LE,5,2010-11-16,great variety,excellent selection coffee help decide type fl...,1
270563,271264,B001IZHZGS,A3QKH94YUG0IEI,5,2012-08-27,Great,great fresh came fast love gum taste like hawa...,1
559088,560441,B0005ZZ4CE,AKYX5JND0LTL8,4,2010-09-14,"Best sauce, Worst price",ortega medium taco sauce best sauce ever entir...,1
136056,136380,B006Q820X0,#oc-R192YGWMM86YIU,5,2012-04-25,UPDATED Review Problem resolved,since new product released feedback usually lo...,1


In [19]:
# initialize NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# create get_sentiment function
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment

# apply get_sentiment function
random_sample_df['sentiment'] = random_sample_df['Text'].apply(get_sentiment)
random_sample_df

,Id,ProductId,UserId,Score,Time,Summary,Text,Positive,sentiment
165363,165795,B000CPZSC8,A3CK0SW0H2HGWX,5,2012-02-20,Best Value Vanilla!,vanilla pricing fluctuates pretty low right st...,1,1
312513,313285,B003WAX3H2,A2S3MH8WNMG0LE,5,2010-11-16,great variety,excellent selection coffee help decide type fl...,1,1
270563,271264,B001IZHZGS,A3QKH94YUG0IEI,5,2012-08-27,Great,great fresh came fast love gum taste like hawa...,1,1
559088,560441,B0005ZZ4CE,AKYX5JND0LTL8,4,2010-09-14,"Best sauce, Worst price",ortega medium taco sauce best sauce ever entir...,1,1
136056,136380,B006Q820X0,#oc-R192YGWMM86YIU,5,2012-04-25,UPDATED Review Problem resolved,since new product released feedback usually lo...,1,1
...,...,...,...,...,...,...,...,...,...
465030,466204,B001E18K8S,A2G9TPDCXH2WJ8,5,2012-09-06,Helps with Heart Palpitations,little bit research learned magnesium play cru...,1,1
184192,184685,B000VIHZSQ,A3Q4RJ1C1XYRSR,3,2012-02-27,"The Tuna and Salmon are Awesome, the cans are ...",eating product last year excellent however can...,1,1
79889,80054,B001HTG4TO,A1BRTFJFISZ1V4,5,2010-03-08,Love It,boil water add cereal simmer min remove heat a...,1,1
99105,99340,B0009ETA76,A2DBEJ3JUJ4TPU,5,2011-03-12,It worked! But I smelled.,towards end breastfeeding son began taking fen...,1,1


In [20]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(random_sample_df['Positive'], random_sample_df['sentiment']))

[[ 302  648]
 [   2 9048]]


In [22]:
from sklearn.metrics import classification_report

print(classification_report(random_sample_df['Positive'], random_sample_df['sentiment']))

              precision    recall  f1-score   support

           0       0.99      0.32      0.48       950
           1       0.93      1.00      0.97      9050

    accuracy                           0.94     10000
   macro avg       0.96      0.66      0.72     10000
weighted avg       0.94      0.94      0.92     10000

